> **Note:** This notebook is intended to be run on **Google Colab**.

### I. Load & Imports

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install ultralytics

In [ ]:
import zipfile
import os
from ultralytics import YOLO
from IPython.display import Image
from PIL import Image as PILImage
from IPython.display import display
import matplotlib.pyplot as plt

### II. Dataset Extraction & Preparation

In [ ]:
# path to the zip dataset
zip_path = "Tire Dataset.v2i.yolov8 (1).zip"

# directory where the dataset will be extracted
extract_path = "/content/dataset"

# create the directory if it does not exist
os.makedirs(extract_path, exist_ok=True)

# extract all files from the zip archive
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction terminée. Les fichiers ont été extraits dans :", extract_path)

In [ ]:
# rename the "valid" folder to "val"
os.rename('/content/dataset/valid', '/content/dataset/val')

# read the content of a label file
with open('/content/dataset/train/labels/1001_png.rf.05300e4f0f2316063d99d0ca39a76ca6.txt') as f:
    lines = f.readlines()
    print("Contenu du fichier :")
    print(lines)

# extract class IDs from the label file
class_ids = [line.strip().split()[0] for line in lines]

# list unique class IDs used
unique_ids = sorted(set(class_ids))
print("\nClasses utilisées dans ce fichier :", unique_ids)


### III. YAML Configuration

In [ ]:
# create and write the corrected YAML configuration file
fixed_yaml = """
train: ../train/images
val: ../val/images
test: ../test/images

nc: 1
names: ['car-tire']

roboflow:
  workspace: iotml
  project: tire-dataset
  version: 2
  license: Public Domain
  url: https://universe.roboflow.com/iotml/tire-dataset/dataset/2
"""

# save the YAML file to the dataset directory
with open('/content/dataset/data.yaml', 'w') as f:
    f.write(fixed_yaml)

### IV. Model Training & Validation

### Explication du modèle et des paramètres d’entraînement

Modèle utilisé : **YOLOv8m** - version intermédiaire de la famille YOLOv8, offrant un bon compromis entre vitesse et précision. 
Principaux paramètres: 
- Transfert d’apprentissage
  - Chargement de poids pré-entraînés sur COCO
  - Accélèration de la convergence et amélioration de la précision initiale

- Automatic Mixed Precision (AMP)
  - AMP combine des calculs en 16 bits et 32 bits
  - Avantages : vitesse accrue et réduction de la mémoire GPU

- Chargement des données
  - Scan et mise en cache des images et labels
  - Dataset utilisé :
    - Train : 1461 images
    - Validation : 190 images
  - Résolution utilisée : 800×800 px, optimisée pour YOLOv8m (meilleure précision que 640×640)

- Augmentations 
Transformations légères pour la robustesse du modèle :
  - `Blur(p=0.01)`
  - `MedianBlur(p=0.01)`
  - `ToGray(p=0.01)`
  - Amélioration de généralisation du modèle

- Optimiseur (AdamW)
  - Sélection automatique de  l’optimiseur
  - Paramètres importants :
    - `lr = 0.002`
    - `momentum = 0.9`
    - `weight_decay` activé (meilleure régularisation)
  - Bon équilibre entre stabilité et généralisation

- Chargement en parallèle
  - Utilisation de 2 workers pour accélérer la préparation des batches

- Début de l’entraînement
Entraînement pendant 50 epochs et optimisation : 
  - box_loss – précision de la localisation
  - cls_loss – classification des objets
  - dfl_loss – Distribution Focal Loss (forme/uniformité des boîtes)

Une diminution progressive de ces pertes indique un modèle qui apprend correctement.

In [ ]:
# Load the YOLOv8 medium model
model = YOLO("yolov8m.pt")

# Train the model using a larger image size (800px) to improve detection precision
model.train(data="/content/dataset/data.yaml", epochs=50, imgsz=800)

### V. Visualization of Model Predictions

In [ ]:
def display_img(path, width=800, title=None):
    """
    Display a single image with optional resizing and custom title.
    """
    img = PILImage.open(path)
    img = img.resize((width, int(img.height * width / img.width)))  # keep aspect ratio

    if title:
        print(f"{title}")

    display(img)


def display_img_list(folder, files, width=800):
    """
    Display a list of images from a given folder.
    """
    for filename, title in files:
        full_path = f"{folder}/{filename}"
        display_img(full_path, width=width, title=title)


In [ ]:
# List of training performance plots generated by YOLOv8.
train_files = [
    ("results.png", "Training Metrics Overview"),
    ("BoxPR_curve.png", "Box Precision-Recall Curve (Train)"),
    ("BoxF1_curve.png", "Box F1-Confidence Curve (Train)"),
    ("BoxP_curve.png", "Box Precision Curve (Train)"),
    ("BoxR_curve.png", "Box Recall Curve (Train)"),
    ("confusion_matrix.png", "Confusion Matrix (Train)"),
    ("labels.jpg", "Label Distribution"),
]

display_img_list("runs/detect/train", train_files)

Le modèle présente d'excellentes performances.
Les courbes d’évaluation montrent une précision et un rappel élevés, avec un score F1 optimal d’environ 0,91 atteint pour un seuil de confiance proche de 0,32.
Les métriques mAP50 et mAP50-95 indiquent également une bonne qualité de localisation et de classification, confirmant la robustesse du modèle.

### VI. Model Testing

In [ ]:
# Load the trained YOLOv8 model using the best checkpoint from training
model = YOLO("runs/detect/train/weights/best.pt")

# Evaluate the model on the test split defined in data.yaml
metrics = model.val(data="/content/dataset/data.yaml", split="test")


In [ ]:
# List of evaluation plots generated by YOLOv8 for the test dataset.
test_files = [
    ("BoxF1_curve.png", "Box F1-Confidence Curve (Test)"),
    ("BoxPR_curve.png", "Box Precision-Recall Curve (Test)"),
    ("BoxP_curve.png", "Box Precision Curve (Test)"),
    ("BoxR_curve.png", "Box Recall Curve (Test)"),
    ("confusion_matrix.png", "Confusion Matrix (Test)"),
]

display_img_list("runs/detect/val", test_files)

Les métriques obtenues sur les 103 images du jeu de test montrent que le modèle réalise une détection très solide :
- Précision : 0.914
→ Très peu de faux positifs : lorsque le modèle détecte un pneu, il se trompe rarement.

- Recall : 0.909
→ Très bonne couverture des objets : le modèle détecte la majorité des pneus présents.

- mAP@50 : 0.953
→ Excellent taux de correspondance IoU ≥ 0.50 entre prédictions et annotations.

- mAP@50–95 : 0.609
→ Indique une performance correcte mais perfectible pour la localisation fine (IoU plus stricts).

Ces résultats confirment la robustesse et la généralisation du modèle ( pas d'overfitting visible).

Rséultats du score F1:
- Score F1 maximal ≈ 0.91 atteint à une confiance d’environ 0.14
- Optimisation de l'équilibre par un seuil de confiance bas

- Excellente stabilité de la courbe entre 0.1 et 0.6 : 
→ Le modèle reste performant dans une large plage de seuils de confiance.
- Hausse des faux positifs en dessous de 0.05

### VII. Qualitative Evaluation (Visual Comparison)

In [ ]:
def display_side_by_side(img1_path, img2_path, label1="Ground Truth", label2="Prediction"):
    """
    Display two images side by side for visual comparison.

    Parameters:
        img1_path (str): Path to the first image (typically ground-truth labels).
        img2_path (str): Path to the second image (typically model predictions).
        label1 (str): Title displayed above the first image.
        label2 (str): Title displayed above the second image.
    """

    # Load both images
    img1 = PILImage.open(img1_path)
    img2 = PILImage.open(img2_path)

    # Create a figure with two columns
    fig, axs = plt.subplots(1, 2, figsize=(14, 7))

    # Left image (Ground Truth)
    axs[0].imshow(img1)
    axs[0].set_title(label1, fontsize=14)
    axs[0].axis('off')

    # Right image (Prediction)
    axs[1].imshow(img2)
    axs[1].set_title(label2, fontsize=14)
    axs[1].axis('off')

    # Adjust spacing and display
    plt.tight_layout()
    plt.show()


# Display the first batch from validation: ground truth vs prediction
display_side_by_side(
    "runs/detect/val/val_batch0_labels.jpg",
    "runs/detect/val/val_batch0_pred.jpg"
)
